In [2]:
import cv2
import numpy as np
import os
import random
import pandas as pd
from skimage import io, transform
from PIL import Image
from matplotlib import pyplot as plt
from scipy.cluster.vq import kmeans, vq
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import joblib
from sklearn.metrics import confusion_matrix, accuracy_score #sreeni


# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
# import os, os.path
# import random
# from skimage import io, transform
# import torchvision.models as models
# from PIL import Image
# import matplotlib
# import matplotlib.pyplot as plt
# import statistics
# from scipy import signal

# from data_providers import CatDogDataset, test_model
%matplotlib inline

In [3]:
# data_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
#     ])

set_list = ['A', 'B', 'C']
batch_size = 16
criterion = nn.CrossEntropyLoss()
root_dir = '../catdog/'

# use gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
def testing(root_dir, class_name, pert_list, set_list, batch_size, device, criterion):
    
    acc_mean_list = []
    acc_std_list = []
    for p in pert_list:
        
        acc_list = []
        for testset in set_list:
            set_name = testset + '_test'
            
            image_dataset = class_name(root_dir, 
                                set_name,
                                data_transform,
                                p)
            dataloader = DataLoader(image_dataset, 
                                 batch_size = batch_size, 
                                 shuffle=True, 
                                 num_workers=2)
            dataset_size = len(image_dataset)
            
            model = models.resnet18()
            model.fc = nn.Linear(512, 2)
            model_path = './models/' + testset + '_astest'
            model.load_state_dict(torch.load(model_path))
            model = model.to(device)
            
            acc = test_model(model, dataloader,
                              dataset_size, device,
                              criterion)
            acc_list.append(acc.item())
            
        acc_mean = statistics.mean(acc_list)
        acc_std = statistics.stdev(acc_list)
        
        acc_mean_list.append(acc_mean)
        acc_std_list.append(acc_std)
        
    return acc_mean_list, acc_std_list

# 1. Gaussian pixel noise

In [5]:
class GPN(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, std):
        super().__init__(root_dir, set_name, transform)
        self.std = std
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
            
        noise = np.rint(np.random.normal(0, self.std, (224, 224, 3))).astype(np.uint8)
        img = np.clip(img + noise, 0, 255)

        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [6]:
std_list = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

acc_mean_list, acc_std_list = testing(root_dir, GPN, std_list, set_list, 
            batch_size, device, criterion)

plt.errorbar(std_list, acc_mean_list, acc_std_list)
plt.xticks(std_list)
plt.xlabel('standard deviation')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/gpn.png')   
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './models/A_astest'

# 2. Gaussian blurring

In [ ]:
class GB(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, convolving_times):
        super().__init__(root_dir, set_name, transform)
        self.convolving_times = convolving_times
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
            
            
        mask = 1/16 * np.array([[1, 2, 1],
                                [2, 4, 2],
                                [1, 2, 1]])
      
        for n in range(self.convolving_times):
            for c in range(3):
                img[:, :, c] = signal.correlate2d(img[:,:,c], mask, 
                                                 boundary='fill', mode='same')
                
        img = np.clip(img, 0, 255)

        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [ ]:
convolving_list = [x for x in range(10)]



acc_mean_list, acc_std_list = testing(root_dir, GB, convolving_list, set_list, 
            batch_size, device, criterion)

plt.errorbar(convolving_list, acc_mean_list, acc_std_list)
plt.xticks(convolving_list)
plt.xlabel('convolving time')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/gb.png')   
plt.show()

# 3. Image contrast increase

In [ ]:
class ICI(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, contrast):
        super().__init__(root_dir, set_name, transform)
        self.contrast = contrast
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
            
      
        img = np.rint(self.contrast * img).astype(np.uint8)
                
        img = np.clip(img, 0, 255)

        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [ ]:
contrast = [1.0, 1.03, 1.06, 1.09, 1.12, 1.15, 
                   1.18, 1.21, 1.24, 1.27]

# use gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


acc_mean_list, acc_std_list = testing(root_dir, ICI, contrast, set_list, 
            batch_size, device, criterion)

plt.errorbar(contrast, acc_mean_list, acc_std_list)
plt.xticks(contrast)
plt.xlabel('contrast')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/ici.png')   
plt.show()

# 4. Image contrast decrease

In [ ]:
contrast = [1.0, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10]

# use gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


acc_mean_list, acc_std_list = testing(root_dir, ICI, contrast, set_list, 
            batch_size, device, criterion)

plt.errorbar(contrast, acc_mean_list, acc_std_list)
plt.xticks(contrast)
plt.xlim(1.1, 0.0)
plt.xlabel('contrast')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/icd.png')   
plt.show()

# 5. Image brightness increase

In [ ]:
class IB(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, brightness_increase):
        super().__init__(root_dir, set_name, transform)
        self.brightness_increase = brightness_increase
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
            
      
        img = (img + self.brightness_increase).astype(np.uint8)
                
        img = np.clip(img, 0, 255)

        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [ ]:
brightness_increase = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

# use gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


acc_mean_list, acc_std_list = testing(root_dir, IB, brightness_increase, set_list, 
            batch_size, device, criterion)

plt.errorbar(brightness_increase, acc_mean_list, acc_std_list)
plt.xticks(brightness_increase)
plt.xlabel('brightness_increase')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/ibi.png')   
plt.show()

# 6. Image brightness decrease

In [ ]:
brightness_increase = [0, -5, -10, -15, -20, -25, -30, -35, -40, -45]

# use gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


acc_mean_list, acc_std_list = testing(root_dir, IB, brightness_increase, set_list, 
            batch_size, device, criterion)

plt.errorbar(brightness_increase, acc_mean_list, acc_std_list)
plt.xticks(brightness_increase)
plt.xlim(5, -50)
plt.xlabel('brightness_decrease')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/ibd.png')   
plt.show()

# 7. HSV Hue noise increase

In [ ]:
class HNI(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, std):
        super().__init__(root_dir, set_name, transform)
        self.std = std
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
        
        
        img = matplotlib.colors.rgb_to_hsv(img/255)

        noise = np.random.normal(0, self.std, (224, 224))
        img[:, :, 0] = img[:, :, 0] + noise

        img[:, :, 0] = np.where(img[:, :, 0] > 1, img[:, :, 0] - 1,  img[:, :, 0])
        img[:, :, 0] = np.where(img[:, :, 0] < 0, img[:, :, 0] + 1,  img[:, :, 0])

        img = matplotlib.colors.hsv_to_rgb(img)
        img = np.rint(img * 255).astype(np.uint8)
        
        

        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [ ]:
std_list = [0.00, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18]



acc_mean_list, acc_std_list = testing(root_dir, HNI, std_list, set_list, 
            batch_size, device, criterion)

plt.errorbar(std_list, acc_mean_list, acc_std_list)
plt.xticks(std_list)
plt.xlabel('std')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/hni.png')   
plt.show()

# 8. HSV Saturation noise increase

In [ ]:
class SNI(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, std):
        super().__init__(root_dir, set_name, transform)
        self.std = std
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
        
        
        img = matplotlib.colors.rgb_to_hsv(img/255)

        noise = np.random.normal(0, self.std, (224, 224))
        img[:, :, 1] = img[:, :, 1] + noise
        
        img = np.clip(img, 0.0, 1.0)


        img = matplotlib.colors.hsv_to_rgb(img)
        img = np.rint(img * 255).astype(np.uint8)
        
        

        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [ ]:
std_list = [0.00, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18]



acc_mean_list, acc_std_list = testing(root_dir, SNI, std_list, set_list, 
            batch_size, device, criterion)

plt.errorbar(std_list, acc_mean_list, acc_std_list)
plt.xticks(std_list)
plt.xlabel('std')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/sni.png')   
plt.show()

# 9. Occlusion of the image increase

In [ ]:
class OII(CatDogDataset):
    def __init__(self, root_dir, set_name, transform, edge_length):
        super().__init__(root_dir, set_name, transform)
        self.edge_length = edge_length
    
    def __getitem__(self, index):

        image_name, target = self.data[index][0], self.data[index][1]
        img = io.imread(self.root_dir + image_name)

    
        if len(img.shape) == 2:
            img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
        
        # randomly choose the top-left pixel of the square region
        
        
        x = random.randint(0, 224-self.edge_length)
        y = random.randint(0, 224-self.edge_length)

        img[x:x+self.edge_length, y: y+self.edge_length, :] = 0
        
        img = Image.fromarray(img)
        
        
        if self.transform:
            img = self.transform(img)

        return img, np.array([target])

In [ ]:
length_list = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]



acc_mean_list, acc_std_list = testing(root_dir, OII, length_list, set_list, 
            batch_size, device, criterion)

plt.errorbar(length_list, acc_mean_list, acc_std_list)
plt.xticks(length_list)
plt.xlabel('length')
plt.ylabel('accuracy')
plt.savefig('./robustness_graphs/oii.png')   
plt.show()